In [13]:
import os
import pdfplumber
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import csv
print('the Cell was executed without error')

the Cell was executed without error


In [18]:
# Load the job description from a PDF
with pdfplumber.open(r"C:\Users\kabir\Desktop\Projects\Resumes_Ranking\data\raw\Data\JobDescription\10399912.pdf") as pdf:
    job_description_text = ""
    for page in pdf.pages:
        job_description_text += page.extract_text()
# Process the job description text
stop_words = stopwords.words('english') 
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
job_description_tfidf = tfidf_vectorizer.fit_transform([job_description_text])

In [19]:
#print(job_description_tfidf.shape)
#job_description_tfidf
#job_description_text

In [33]:
# Folder containing resume files
resume_folder = r"C:\Users\kabir\Desktop\Projects\Resumes_Ranking\data\raw\Data\HR"
# Process each resume and calculate cosine similarity
similarities = {}
for resume_file in os.listdir(resume_folder):
    if resume_file.endswith('.pdf'):
        try:
            with pdfplumber.open(os.path.join(resume_folder, resume_file)) as pdf:
                resume_text = ""
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        resume_text += page_text  # Append page text if not empty
                
                if resume_text.strip():  # Ensure extracted text is not empty
                    # Process the resume text
                    resume_tfidf = tfidf_vectorizer.transform([resume_text])
                    
                    # Calculate cosine similarity
                    similarity = cosine_similarity(job_description_tfidf, resume_tfidf)
                    similarities[resume_file] = similarity[0][0]
                    print(similarities[resume_file])
                
                else:
                    print(f"Warning: Empty text in {resume_file}")
        except Exception as e:
            print(f"Error processing {resume_file}: {e}")

# Sort and rank resumes based on cosine similarity
sorted_resumes = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

# Specify the CSV output file path in Kaggle's working directory
output_csv_file = 'similarity_scores.csv'

# Write the results to the CSV file
with open(output_csv_file, mode='w', newline='') as csv_file:
    fieldnames = ['Rank', 'Resume File', 'Cosine Similarity']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for idx, (resume_file, similarity) in enumerate(sorted_resumes, start=1):
        writer.writerow({'Rank': idx, 'Resume File': resume_file, 'Cosine Similarity': similarity})

print(f"Results saved to {output_csv_file}")

0.9999999999999989
0.44110592258526726
0.43056401330681876
0.5860755334317378
0.5408665275774263
Results saved to similarity_scores.csv


In [ ]:
#print(resume_text)